In [79]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb

In [80]:
from sklearn.metrics import mean_absolute_error


In [81]:
df_NTC = pd.read_csv("../../data/NTC.csv")
df_losses = pd.read_csv("../../data/Avtice-losses.csv")

df_renewable = pd.read_csv("../../data/Forecast-renewable-generation.csv")
df_temp = pd.read_csv("../../data/Forecast-temperature.csv")

df_losses = df_losses.drop(df_losses.index[0])

In [82]:
#print(df_NTC.columns)
print(df_losses.columns)
#print(df_renewable.columns)
print(df_temp.columns)

Index(['Unnamed: 0', 'Wirkverluste/Active Losses'], dtype='object')
Index(['datetime', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it'],
      dtype='object')


In [83]:
df_losses = df_losses.rename(columns={
    'Unnamed: 0': 'datetime',
})

In [84]:

df_NTC['datetime'] = pd.to_datetime(df_NTC['datetime'])
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])
df_renewable['datetime'] = pd.to_datetime(df_renewable['datetime'])
df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])

In [85]:
df_losses['Wirkverluste/Active Losses'] = pd.to_numeric(df_losses['Wirkverluste/Active Losses'], errors='coerce')

In [86]:
# Convert "Zeitstempel" column to datetime format, if not already
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])

# Subtract 15 minutes from each date in the "Zeitstempel" column
df_losses['datetime'] = df_losses['datetime'] - pd.Timedelta(minutes=15)



In [87]:

df_losses_new = df_losses.set_index('datetime')

hourly_loss = df_losses_new.resample('1H', closed='left', label='left').sum()

In [88]:
hourly_loss["Wirkverluste/Active Losses"]=hourly_loss["Wirkverluste/Active Losses"]/1000


In [89]:
#hourly_loss

In [90]:


# Check if the data is already in 6-hour intervals
if (df_temp['datetime'].diff().dt.total_seconds() / 3600).median() == 6:
    # If the data is in 6-hour intervals, leave the datetime column as it is
    hourly_temp = df_temp
else:
    # If the data is not in 6-hour intervals, resample it to hourly intervals
    hourly_temp = df_temp.resample('H', on='datetime').mean().reset_index()

    # Fill missing values in other columns with the values from the initial 6-hourly data
    columns_to_fill = hourly_temp.columns.difference(['datetime'])
    hourly_temp[columns_to_fill] = hourly_temp[columns_to_fill].fillna(method='ffill', limit=6)
    # Replace '6' with the appropriate number if you want to fill more or fewer hours

# Print the resulting hourly data
display(hourly_temp.head(10))

,datetime,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
0,2019-01-01 01:00:00,4.1067,5.9729,7.4268,4.0281
1,2019-01-01 02:00:00,4.1067,5.9729,7.4268,4.0281
2,2019-01-01 03:00:00,4.1067,5.9729,7.4268,4.0281
3,2019-01-01 04:00:00,4.1067,5.9729,7.4268,4.0281
4,2019-01-01 05:00:00,4.1067,5.9729,7.4268,4.0281
5,2019-01-01 06:00:00,4.1067,5.9729,7.4268,4.0281
6,2019-01-01 07:00:00,1.7595,5.5398,6.3425,2.7281
7,2019-01-01 08:00:00,1.7595,5.5398,6.3425,2.7281
8,2019-01-01 09:00:00,1.7595,5.5398,6.3425,2.7281
9,2019-01-01 10:00:00,1.7595,5.5398,6.3425,2.7281


In [91]:
print(df_NTC.columns)
print(hourly_loss.columns)
print(df_renewable.columns)
print(hourly_temp.columns)

Index(['datetime', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH',
       'FR_CH', 'IT_CH'],
      dtype='object')
Index(['Wirkverluste/Active Losses'], dtype='object')
Index(['datetime', 'solar_fore_de [MW]', 'solar_fore_it [MW]',
       'wind_fore_de [MW]', 'wind_fore_it [MW]'],
      dtype='object')
Index(['datetime', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it'],
      dtype='object')


In [92]:
hourly_loss = hourly_loss.reset_index()

In [93]:
print(hourly_loss)

                 datetime  Wirkverluste/Active Losses
0     2019-01-01 00:00:00                  139.525004
1     2019-01-01 01:00:00                  129.716036
2     2019-01-01 02:00:00                  133.398074
3     2019-01-01 03:00:00                  135.133852
4     2019-01-01 04:00:00                  131.699424
...                   ...                         ...
26299 2021-12-31 19:00:00                  171.707318
26300 2021-12-31 20:00:00                  159.462903
26301 2021-12-31 21:00:00                  155.109520
26302 2021-12-31 22:00:00                  171.370277
26303 2021-12-31 23:00:00                  146.054791

[26304 rows x 2 columns]


In [94]:
merged_df = df_NTC.merge(hourly_loss, on='datetime', how='outer')
merged_df = merged_df.merge(df_renewable, on='datetime', how='outer')
merged_df = merged_df.merge(hourly_temp, on='datetime', how='outer')

In [95]:
na_counts_per_column = merged_df.isna().sum()

In [96]:
na_counts_per_column 

datetime                         0
CH_AT                            3
CH_DE                            3
CH_FR                            3
CH_IT                            3
AT_CH                            3
DE_CH                            3
FR_CH                            3
IT_CH                            3
Wirkverluste/Active Losses       0
solar_fore_de [MW]               3
solar_fore_it [MW]               3
wind_fore_de [MW]                3
wind_fore_it [MW]                3
temperature_fore_ch           1578
temperature_fore_fr           1578
temperature_fore_de           1578
temperature_fore_it           1578
dtype: int64

In [97]:
merged_df_incomplete = merged_df.dropna()

In [98]:
merged_df_incomplete.columns

Index(['datetime', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH',
       'FR_CH', 'IT_CH', 'Wirkverluste/Active Losses', 'solar_fore_de [MW]',
       'solar_fore_it [MW]', 'wind_fore_de [MW]', 'wind_fore_it [MW]',
       'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
       'temperature_fore_it'],
      dtype='object')

In [99]:
#merged_df_incomplete.reset_index(inplace=True)
#merged_df_incomplete=merged_df_incomplete.drop('datetime')

In [100]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'Wirkverluste/Active Losses': 'Loss'})

In [101]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_de [MW]': 'solar_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_it [MW]': 'solar_fore_it'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_de [MW]': 'wind_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_it [MW]': 'wind_fore_it'})
#drop the datetime

#merged_df_incomplete=merged_df_incomplete.drop('datetime')


In [103]:
merged_df_incomplete = merged_df_incomplete.reset_index()

In [105]:
merged_df_incomplete = merged_df_incomplete.drop(columns=['datetime'])


In [106]:
merged_df_incomplete

,index,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,Loss,solar_fore_de,solar_fore_it,wind_fore_de,wind_fore_it,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
0,1,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,129.716036,0.0,0.0,23052.3310,4596.5916,4.1067,5.9729,7.4268,4.0281
1,2,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,133.398074,0.0,0.0,24969.9701,4478.5564,4.1067,5.9729,7.4268,4.0281
2,3,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,135.133852,0.0,0.0,27082.9626,4323.3712,4.1067,5.9729,7.4268,4.0281
3,4,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,131.699424,0.0,0.0,26890.9717,4231.8283,4.1067,5.9729,7.4268,4.0281
4,5,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,147.391128,0.0,0.0,27740.1555,4266.3082,4.1067,5.9729,7.4268,4.0281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24727,26305,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,171.707318,0.0,0.0,36997.7200,1108.4000,8.5300,9.9600,10.7900,9.5900
24728,26306,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,159.462903,0.0,0.0,35666.9300,1077.9700,8.0000,9.4000,10.6300,9.1100
24729,26307,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,155.109520,0.0,0.0,34383.8800,1048.2800,7.5000,8.8800,10.5100,8.6700
24730,26308,1200.0,4000.0,1400.0,3780.0,1200.0,800.0,3200.0,1810.0,171.370277,0.0,0.0,33075.2500,1078.7800,6.9700,8.5100,10.3200,8.1400


In [76]:
merged_ready= merged_df_incomplete

In [50]:
features = ['CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
           'solar_fore_de', 'solar_fore_it', 'wind_fore_de', 'wind_fore_it',
           'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
           'temperature_fore_it']

# Create the lagged features
for feature in features:
    for lag in range(1, lag_input+1):
        col_name = f"{feature}_lag{lag}"
        merged_ready[col_name] = merged_ready[feature].shift(lag)
merged_ready = merged_ready.iloc[lag_input:, :]

#merged_df_incomplete.set_index('datetime', inplace=True)
X = merged_df_incomplete.drop('Loss', axis=1)  # Features
y = merged_df_incomplete['Loss']  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'device': 'gpu'  # Enable GPU support
}
# Create the LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
# Train the model
model = lgb.train(params, train_data)

# Predict the test data
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.4f}")


/tmp/ipykernel_28336/4206638143.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_ready[col_name] = merged_ready[feature].shift(lag)


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 21122
[LightGBM] [Info] Number of data points in the train set: 19785, number of used features: 120
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 120 dense feature groups (2.26 MB) transferred to GPU in 0.010270 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 103.173180


In [109]:


def compute_mae(merged_df_ready_prep, lag_input):
    # Ensure the merged_ready DataFrame is created first
    merged_ready = merged_df_ready_prep.copy()

    features = ['CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
               'solar_fore_de', 'solar_fore_it', 'wind_fore_de', 'wind_fore_it',
               'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
               'temperature_fore_it']

    # Create the lagged features
    for feature in features:
        for lag in range(1, lag_input+1):
            col_name = f"{feature}_lag{lag}"
            merged_ready[col_name] = merged_ready[feature].shift(lag)

    merged_ready = merged_ready.iloc[lag_input:, :]

    X = merged_ready.drop('Loss', axis=1)  # Features
    y = merged_ready['Loss']  # Target variable

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'device': 'gpu'  # Enable GPU support
    }

    # Create the LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)
    
    # Train the model
    model = lgb.train(params, train_data)

    # Predict the test data
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)

    return mae



In [111]:
lag_1=compute_mae(merged_ready, 1)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5275
[LightGBM] [Info] Number of data points in the train set: 19784, number of used features: 30
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 30 dense feature groups (0.60 MB) transferred to GPU in 0.002001 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 103.082425


In [ ]:
results = {}  # Dictionary to store the MAE for each lag

for i in range(1, 25):
    mae_value = compute_mae(merged_ready, i)
    results[f"lag_{i}"] = mae_value
    print(f"MAE for lag_{i}: {mae_value:.4f}")

# If you want to access the value later, you can do it like this:
# mae_for_lag_1 = results["lag_1"]


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5275
[LightGBM] [Info] Number of data points in the train set: 19784, number of used features: 30
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 30 dense feature groups (0.60 MB) transferred to GPU in 0.080487 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 103.082425
MAE for lag_1: 14.6845
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7899
[LightGBM] [Info] Number of data points in the train set: 19784, number of used features: 45
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogra

In [47]:
# to do:

In [49]:

# do it for each day 24 forecasts


